# LLM-based Permutation Solver

**Workflow:**
1. Define parameters (provider, model, prompts)
2. Create solver
3. Generate base algorithm via LLM
4. Test base algorithm
5. Optimize algorithm via LLM
6. Test optimized algorithm
7. Test manual (hand-written) algorithm

In [20]:
import importlib
import numpy as np
import main as main_module

importlib.reload(main_module)

from main import (
    PermutationSolver,
    PermutationConfig,
    create_llm_client,
    generate_random_permutation,
    apply_moves,
    execute_generated_code,
    save_code_to_file,
 )

## 1. Parameters

In [21]:
# ----- LLM provider settings -----
PROVIDER = 'claude'
MODEL = 'claude-opus-4-5-20251101'

PROVIDER = 'openai'
MODEL = 'gpt-5.2'
# ----- Permutation settings -----
PERM_LENGTH = 4
NUM_TESTS = 10

# ----- Toggle: use custom prompts or defaults from main.py -----
USE_CUSTOM_PROMPTS = True

In [22]:
# ----- Custom prompts (used only when USE_CUSTOM_PROMPTS = True) -----

CUSTOM_SYSTEM_PROMPT = """You are an expert algorithm designer specializing in combinatorial puzzles and permutation theory.
Your task is to create efficient, constructive algorithms that solve permutation problems using only allowed moves.
You must provide working Python code that is self-contained and can be executed directly.
Always follow the constraints specified in the task and provide polynomial-time solutions."""

CUSTOM_MAIN_PROMPT = """Task: Implement a constructive sorting algorithm that sorts a given vector using ONLY allowed moves (L, R, X).

Input: A vector a of length n (0-indexed) containing distinct integers from 0 to n-1.

Allowed moves:
L: Left cyclic shift — shifts all elements one position to the left, with the first element moving to the end. Example: [0,1,2,3] -> [1,2,3,0].
R: Right cyclic shift — shifts all elements one position to the right, with the last element moving to the beginning. Example: [0,1,2,3] -> [3,0,1,2].
X: Transposition of the first two elements — swaps the elements at positions 0 and 1. Example: [0,1,2,3] -> [1,0,2,3].

CRITICAL CONSTRAINTS:
1. NO BFS, DFS, or any graph search algorithms are allowed
2. The algorithm must run in POLYNOMIAL TIME (O(n^k) for some constant k)
3. No exponential-time algorithms (like brute force search through permutations)
4. Must use a constructive, iterative approach that builds the solution step by step
5. No storing or exploring multiple states simultaneously

Strict operational constraints:
- No other operations, slicing, built-in sorting functions, or creating new arrays are allowed (except for a copy to simulate sorting)
- All moves must be appended to the moves list immediately after performing them (as strings: 'L', 'R', or 'X')
- Applying the sequence of moves sequentially to a copy of the input vector must yield a fully sorted ascending array [0, 1, 2, ..., n-1]
- Moves can be used multiple times as needed
- The algorithm must continue applying moves until the array is fully sorted

ALGORITHMIC REQUIREMENTS:
- Use a constructive approach: develop a strategy that systematically brings elements to their correct positions
- Think in terms of bringing the smallest unsorted element to the front, then \"locking\" it in place
- Consider how L and R can help position elements for X swaps
- The solution should work for any n and have predictable, polynomial-time complexity

Expected approach types (choose one):
1. Adaptation of bubble sort/insertion sort using available moves
2. Strategy of bringing smallest element to front, then second smallest, etc.
3. Any other polynomial-time constructive approach

Implementation requirements:
- Implement a function solve(vector) that returns a tuple (moves, sorted_array):
    - moves: list of strings representing all moves performed (e.g., ['L', 'X', 'R', ...])
    - sorted_array: the final sorted array after applying all moves (as a list)
- Include CLI interface:
    - When script is executed directly, accept vector as command-line argument (parse sys.argv[1] as JSON)
    - Use {default_vector} as fallback if no arg is given
    - Output should be JSON object with keys \"moves\" and \"sorted_array\"
- Include minimal example in main block for quick testing
- Code must be fully self-contained and executable without external dependencies (only sys, json allowed)
- JSON output must always be structured and parseable for automated testing

Example usage:
    python solve_module.py \"[3,1,2,0,4]\"

Example output (for illustration):
{{
    \"moves\": [\"X\", \"L\", \"R\", \"X\"],
    \"sorted_array\": [0,1,2,3,4]
}}

IMPORTANT: Focus on developing a polynomial-time constructive algorithm, NOT graph search.
Provide ONLY the Python code, no explanations before or after."""

OPTIMIZE_PROMPT_TEMPLATE  = """You are given a working Python algorithm that sorts a permutation using L, R, X moves.

Current algorithm (O(n³)):
```python
{algorithm_code}
```

YOUR TASK: Optimize to O(n²), specifically ≤ 0.5·n² + O(n) moves.

═══════════════════════════════════════════════════════════════
CRITICAL INSIGHT #1: CYCLE DECOMPOSITION APPROACH
═══════════════════════════════════════════════════════════════
Don't think "place element k at position k". Instead:
1. Decompose permutation into disjoint cycles
2. Each k-cycle needs exactly k-1 adjacent transpositions
3. Process each cycle by "rotating" elements within the cycle

Example: Permutation [2,3,1,4] has cycle (1→2→3→1)
This cycle needs 2 transpositions to resolve, NOT bubble-sorting each element.

═══════════════════════════════════════════════════════════════
CRITICAL INSIGHT #2: TRUE CHAINING (not fake chaining!)
═══════════════════════════════════════════════════════════════
WRONG (what your current code does):
  swap_at(i): L*i, X, R*i  # Cost: 2i+1 each, independent
  
RIGHT (true chaining for consecutive swaps at positions i < j < k):
  L*i, X, L*(j-i), X, L*(k-j), X, R*k  # Cost: 2k + 3 total!

The key: DON'T rotate back after each swap. Stay rotated and continue.

Concrete example for swaps at positions 2, 5, 7 (n=10):
  WRONG: (L L X R R) + (L L L L L X R R R R R) + (L*7 X R*7) = 5+11+15 = 31 moves
  RIGHT: L L X L L L X L L X R R R R R R R = 2+1+3+1+2+1+7 = 17 moves

═══════════════════════════════════════════════════════════════
CRITICAL INSIGHT #3: DIRECTION OPTIMIZATION
═══════════════════════════════════════════════════════════════
For position i:
- If i < n/2: use R*i X L*i (or chain with R direction)
- If i ≥ n/2: use L*(n-i) X R*(n-i) — going "the other way" is shorter!

This halves the rotation cost for positions in the second half.

═══════════════════════════════════════════════════════════════
CRITICAL INSIGHT #4: POST-PROCESSING
═══════════════════════════════════════════════════════════════
After generating moves, simplify:
- X.X → remove both (X is self-inverse)
- L.R or R.L → remove both
- Collapse L.L.L.R.R → L (net rotation)

═══════════════════════════════════════════════════════════════
CRITICAL INSIGHT #5: You must use chaining for BOTH halves of positions!
═══════════════════════════════════════════════════════════════
- Positions i < n/2: chain using L direction
- Positions i ≥ n/2: chain using R direction (going backwards)

DO NOT process swaps independently in a loop like:
  for pos in positions:
      swap_at(pos)  # WRONG - this is O(n) per swap!

═══════════════════════════════════════════════════════════════
ALGORITHM SKELETON (implement this!)
═══════════════════════════════════════════════════════════════
def solve(vector):
    # 1. Find cycle decomposition of the permutation
    cycles = find_cycles(vector)
    
    # 2. For each non-trivial cycle:
    for cycle in cycles:
        if len(cycle) <= 1: continue
        
        # Determine which adjacent positions need swapping
        # to "unwind" this cycle
        swap_positions = compute_swap_positions(cycle)
        
        # 3. CHAIN the swaps: rotate to first position,
        # then X, rotate to next, X, ..., rotate back
        # Choose L vs R based on whether position < n/2
        
    # 4. Simplify the move sequence
    return simplify(moves), sorted_array

═══════════════════════════════════════════════════════════════
COMPLEXITY ANALYSIS TARGET
═══════════════════════════════════════════════════════════════
- Total transpositions needed: at most n-1 (one per element minus cycles)
- Each transposition via chaining: amortized O(n) rotation cost shared
- With direction optimization: positions contribute ≤ n/2 each
- Result: ≤ 0.5·n² + O(n) total moves

REQUIREMENTS:
- Function signature: solve(vector) -> (moves, sorted_array)
- Must handle all permutations correctly
- No BFS/DFS — constructive algorithm only

Provide ONLY the optimized Python code."""

## 2. Create Solver

In [23]:
client = create_llm_client(PROVIDER, MODEL)

if USE_CUSTOM_PROMPTS:
    config = PermutationConfig(
        length=PERM_LENGTH,
        system_prompt=CUSTOM_SYSTEM_PROMPT,
        main_prompt_template=CUSTOM_MAIN_PROMPT,
        optimize_prompt_template=OPTIMIZE_PROMPT_TEMPLATE
    )
    print("Using CUSTOM prompts")
else:
    config = PermutationConfig(length=PERM_LENGTH)
    print("Using DEFAULT prompts")

solver = PermutationSolver(client, config)
print(f"Solver created with {client.name}")

Created client: OpenAI (gpt-5.2)
Using CUSTOM prompts
Solver created with OpenAI (gpt-5.2)


## 3. Generate Base Algorithm

In [24]:
code = solver.generate_algorithm(PERM_LENGTH)

print("Generated code:")
print("=" * 60)
print(code)
print("=" * 60)

Generating algorithm using OpenAI (gpt-5.2)...
Generated code:
import sys
import json

def solve(vector):
    a = list(vector)  # allowed copy to simulate sorting
    n = len(a)
    moves = []

    def do_L():
        if n <= 1:
            return
        first = a[0]
        i = 0
        while i < n - 1:
            a[i] = a[i + 1]
            i += 1
        a[n - 1] = first
        moves.append("L")

    def do_R():
        if n <= 1:
            return
        last = a[n - 1]
        i = n - 1
        while i > 0:
            a[i] = a[i - 1]
            i -= 1
        a[0] = last
        moves.append("R")

    def do_X():
        if n <= 1:
            return
        a[0], a[1] = a[1], a[0]
        moves.append("X")

    # Constructive sorting using adjacent swaps via rotations:
    # To swap positions k-1 and k:
    #   bring (k-1,k) to front with L^(k-1), do X, then restore with R^(k-1)
    # This simulates bubble sort in O(n^3) moves/time, polynomial and constructive.
    if n <

## 4. Test Base Algorithm

In [25]:
# Test on a specific permutation
test_perm = np.array([1, 2, 3, 0])
result = solver.solve(test_perm)

print(f"Input: {test_perm}")
if result['success']:
    print(f"Moves ({len(result['moves'])}): {result['moves']}")
    print(f"Sorted: {result['sorted_array']}")
    print(f"Correct: {result['verification']['is_correct']}")
else:
    print(f"Failed: {result.get('error', 'Unknown')}")

Input: [1 2 3 0]
Moves (9): ['L', 'L', 'X', 'R', 'R', 'L', 'X', 'R', 'X']
Sorted: [0, 1, 2, 3]
Correct: True


In [26]:
# Test on random permutations
results = solver.test_random(n=PERM_LENGTH, num_tests=NUM_TESTS)

print(f"\nSuccess rate: {results['success_rate']*100:.1f}%")
print(f"Passed: {results['success_count']}/{results['num_tests']}")

Test 1/10: [2, 1, 3, 0]
  ✓ Solved with 10 moves
Test 2/10: [2, 0, 1, 3]
  ✓ Solved with 4 moves
Test 3/10: [2, 0, 3, 1]
  ✓ Solved with 9 moves
Test 4/10: [3, 2, 0, 1]
  ✓ Solved with 13 moves
Test 5/10: [2, 1, 0, 3]
  ✓ Solved with 5 moves
Test 6/10: [3, 1, 0, 2]
  ✓ Solved with 10 moves
Test 7/10: [0, 3, 1, 2]
  ✓ Solved with 8 moves
Test 8/10: [1, 0, 3, 2]
  ✓ Solved with 6 moves
Test 9/10: [1, 3, 2, 0]
  ✓ Solved with 12 moves
Test 10/10: [0, 1, 3, 2]
  ✓ Solved with 5 moves

Success rate: 100.0%
Passed: 10/10


## 5. Optimize Algorithm

In [27]:
optimized_code = solver.optimize_algorithm()

print("Optimized code:")
print("=" * 60)
print(optimized_code)
print("=" * 60)
print(f"\nOriginal length: {len(solver.original_code)} chars")
print(f"Optimized length: {len(optimized_code)} chars")

Optimizing algorithm using OpenAI (gpt-5.2)...
Optimization complete.
Optimized code:
import sys
import json

def solve(vector):
    a = list(vector)
    n = len(a)
    moves = []

    if n <= 1:
        return moves, a

    # We will build the sorted permutation [0..n-1] using only:
    # L: rotate left by 1, R: rotate right by 1, X: swap first two.
    #
    # Key idea:
    # - Decompose permutation into cycles of the mapping position -> value.
    # - Each k-cycle can be resolved with exactly k-1 transpositions of the form (0 <-> x)
    #   using a fixed anchor element in that cycle.
    # - Implement each (0 <-> x) swap as:
    #       rotate to bring position x to front, X, rotate back
    #   but CHAIN multiple swaps without rotating back each time.
    # - Use direction optimization: rotate by min(x, n-x) choosing L or R.

    # --- helpers: apply moves to a (for correctness and to update state) ---
    def do_L():
        if n <= 1:
            return
        first = a[0]
     

## 6. Test Optimized Algorithm

In [28]:
# Test optimized version on the same specific permutation
result_opt = solver.solve(test_perm)

print(f"Input: {test_perm}")
if result_opt['success']:
    print(f"Moves ({len(result_opt['moves'])}): {result_opt['moves']}")
    print(f"Sorted: {result_opt['sorted_array']}")
    print(f"Correct: {result_opt['verification']['is_correct']}")
else:
    print(f"Failed: {result_opt.get('error', 'Unknown')}")

Input: [1 2 3 0]
Moves (7): ['L', 'L', 'X', 'R', 'X', 'R', 'X']
Sorted: [0, 1, 2, 3]
Correct: True


In [29]:
# Random tests on optimized version
results_opt = solver.test_random(n=PERM_LENGTH, num_tests=NUM_TESTS)

print(f"\nSuccess rate: {results_opt['success_rate']*100:.1f}%")
print(f"Passed: {results_opt['success_count']}/{results_opt['num_tests']}")

Test 1/10: [2, 0, 1, 3]
  ✓ Solved with 4 moves
Test 2/10: [2, 0, 3, 1]
  ✓ Solved with 7 moves
Test 3/10: [0, 2, 1, 3]
  ✓ Solved with 3 moves
Test 4/10: [2, 0, 3, 1]
  ✓ Solved with 7 moves
Test 5/10: [2, 3, 0, 1]
  ✓ Solved with 10 moves
Test 6/10: [3, 1, 0, 2]
  ✓ Solved with 10 moves
Test 7/10: [3, 1, 0, 2]
  ✓ Solved with 10 moves
Test 8/10: [3, 0, 1, 2]
  ✓ Solved with 7 moves
Test 9/10: [2, 0, 3, 1]
  ✓ Solved with 7 moves
Test 10/10: [3, 1, 0, 2]
  ✓ Solved with 10 moves

Success rate: 100.0%
Passed: 10/10


## 7. Test Manual (Hand-Written) Algorithm

In [15]:
# Paste your own algorithm code here
MANUAL_CODE = '''
import sys
import json

def solve(vector):
    """
    Sorts a vector using only L, R, and X moves.
    
    Optimized O(n²) algorithm using:
    1. Cycle decomposition
    2. Chained swaps with direction optimization
    3. Post-processing to simplify moves
    """
    n = len(vector)
    if n <= 1:
        return ([], list(vector))
    
    if n == 2:
        if vector[0] > vector[1]:
            return (['X'], [vector[1], vector[0]])
        return ([], list(vector))
    
    # Work on a copy
    a = list(vector)
    moves = []
    
    def do_L():
        first = a[0]
        for i in range(n - 1):
            a[i] = a[i + 1]
        a[n - 1] = first
        moves.append('L')
    
    def do_R():
        last = a[n - 1]
        for i in range(n - 1, 0, -1):
            a[i] = a[i - 1]
        a[0] = last
        moves.append('R')
    
    def do_X():
        a[0], a[1] = a[1], a[0]
        moves.append('X')
    
    def find_cycles(arr):
        """Find cycle decomposition of permutation."""
        visited = [False] * n
        cycles = []
        
        for start in range(n):
            if visited[start]:
                continue
            
            cycle = []
            pos = start
            while not visited[pos]:
                visited[pos] = True
                cycle.append(pos)
                pos = arr[pos]  # Follow where element should go
            
            if len(cycle) > 1:
                cycles.append(cycle)
        
        return cycles
    
    def get_swap_positions_for_cycle(cycle):
        """
        For a cycle, determine which adjacent positions need swapping.
        A cycle (p0 -> p1 -> p2 -> ... -> pk -> p0) means:
        - Element at p0 should go to p1
        - Element at p1 should go to p2
        - etc.
        
        We resolve by swapping adjacent pairs to bubble elements into place.
        """
        if len(cycle) <= 1:
            return []
        
        # Sort cycle positions to process from left to right or right to left
        sorted_positions = sorted(cycle)
        swaps = []
        
        # For each element in the cycle (except the last), we need to move it
        # This creates a sequence of adjacent transpositions
        for i in range(len(cycle) - 1):
            # Find where element cycle[i] currently is and where it needs to go
            src = cycle[i]
            dst = cycle[i + 1] if i + 1 < len(cycle) else cycle[0]
            
            # Generate swaps to move element from src to dst
            if src < dst:
                for pos in range(src, dst):
                    swaps.append(pos)
            else:
                for pos in range(src - 1, dst - 1, -1):
                    swaps.append(pos)
        
        return swaps
    
    def execute_chained_swaps(swap_positions):
        """
        Execute swaps at given positions using chaining.
        Split into two groups: positions < n/2 (use L direction) 
        and positions >= n/2 (use R direction).
        """
        if not swap_positions:
            return
        
        # Group swaps by direction preference
        left_swaps = []  # positions where L direction is better (i < n/2)
        right_swaps = []  # positions where R direction is better (i >= n/2)
        
        for pos in swap_positions:
            if pos < n // 2:
                left_swaps.append(pos)
            else:
                right_swaps.append(pos)
        
        # Execute left-group swaps using L direction (chain them)
        if left_swaps:
            # Sort ascending to chain efficiently with L moves
            left_swaps.sort()
            
            current_offset = 0
            for pos in left_swaps:
                # Rotate to bring position 'pos' to position 0
                needed_L = (pos - current_offset) % n
                for _ in range(needed_L):
                    do_L()
                current_offset = (current_offset + needed_L) % n
                
                do_X()
            
            # Rotate back to original alignment
            if current_offset > 0:
                for _ in range(current_offset):
                    do_R()
        
        # Execute right-group swaps using R direction (chain them)
        if right_swaps:
            # Sort descending to chain efficiently with R moves
            right_swaps.sort(reverse=True)
            
            current_offset = 0
            for pos in right_swaps:
                # With R moves, position 0 moves to position 1, etc.
                # To bring position pos to position 0, we need (n - pos) R moves
                # But we track offset differently
                target_rotations = (n - pos + current_offset) % n
                for _ in range(target_rotations):
                    do_R()
                current_offset = (current_offset + target_rotations) % n
                
                do_X()
            
            # Rotate back
            if current_offset > 0:
                for _ in range(current_offset):
                    do_L()
    
    def selection_sort_optimized():
        """
        Selection sort variant with chained operations and direction optimization.
        For each position i, find the minimum in a[i:] and swap it into place.
        """
        for i in range(n - 1):
            # Find minimum in a[i:]
            min_idx = i
            for j in range(i + 1, n):
                if a[j] < a[min_idx]:
                    min_idx = j
            
            if min_idx == i:
                continue
            
            # Bubble the minimum from min_idx to position i using adjacent swaps
            # Collect all swap positions
            swap_positions = list(range(min_idx - 1, i - 1, -1))
            
            if not swap_positions:
                continue
            
            # Execute swaps with chaining
            # All positions from i to min_idx-1, going right to left
            # Use direction optimization based on position
            
            # Chain these swaps
            current_offset = 0
            for pos in swap_positions:
                # Choose direction based on effective position
                effective_pos = (pos - current_offset) % n
                
                if effective_pos <= n // 2:
                    # Use L direction
                    for _ in range(effective_pos):
                        do_L()
                    current_offset = (current_offset + effective_pos) % n
                else:
                    # Use R direction
                    r_moves = n - effective_pos
                    for _ in range(r_moves):
                        do_R()
                    current_offset = (current_offset - r_moves) % n
                
                do_X()
            
            # Rotate back to alignment
            if current_offset > 0:
                for _ in range(current_offset):
                    do_R()
            elif current_offset < 0:
                for _ in range(-current_offset):
                    do_L()
    
    # Use the optimized selection sort
    selection_sort_optimized()
    
    # Simplify moves
    def simplify(move_list):
        """Remove canceling pairs: XX, LR, RL"""
        result = []
        for m in move_list:
            if result:
                last = result[-1]
                if (m == 'X' and last == 'X') or \
                   (m == 'L' and last == 'R') or \
                   (m == 'R' and last == 'L'):
                    result.pop()
                    continue
            result.append(m)
        return result
    
    # Multiple simplification passes
    simplified = moves
    for _ in range(3):
        new_simplified = simplify(simplified)
        if len(new_simplified) == len(simplified):
            break
        simplified = new_simplified
    
    return (simplified, list(a))


def main():
    if len(sys.argv) > 1:
        try:
            vector = json.loads(sys.argv[1])
        except json.JSONDecodeError:
            vector = [3, 2, 0, 1]
    else:
        vector = [3, 2, 0, 1]
    
    moves, sorted_array = solve(vector)
    
    result = {
        "moves": moves,
        "sorted_array": sorted_array
    }
    
    print(json.dumps(result))


if __name__ == "__main__":
    main()
'''

In [16]:
# Test on a specific vector
test_vector = [1, 2, 3, 4, 5, 6, 8, 7, 0]
result_manual = execute_generated_code(MANUAL_CODE, test_vector)

print(f"Input: {test_vector}")
if result_manual['success']:
    print(f"Output: {result_manual['sorted_array']}")
    print(f"Moves ({len(result_manual['moves'])}): {result_manual['moves']}")
else:
    print(f"Error: {result_manual.get('error', 'Unknown')}")

Input: [1, 2, 3, 4, 5, 6, 8, 7, 0]
Output: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Moves (27): ['R', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'R', 'X', 'R', 'R', 'R', 'R', 'R', 'R', 'R']


In [17]:
# Run random tests on the manual algorithm
manual_results = PermutationSolver.test_manual_code_random(
    MANUAL_CODE, n=PERM_LENGTH, num_tests=NUM_TESTS
)

print(f"\nSuccess rate: {manual_results['success_rate']*100:.1f}%")
print(f"Passed: {manual_results['success_count']}/{manual_results['num_tests']}")

Test 1/10: [2, 1, 3, 0]
  ✓ Solved with 10 moves
Test 2/10: [1, 2, 3, 0]
  ✓ Solved with 7 moves
Test 3/10: [1, 2, 0, 3]
  ✓ Solved with 4 moves
Test 4/10: [1, 2, 3, 0]
  ✓ Solved with 7 moves
Test 5/10: [3, 0, 1, 2]
  ✓ Solved with 7 moves
Test 6/10: [1, 2, 3, 0]
  ✓ Solved with 7 moves
Test 7/10: [1, 2, 3, 0]
  ✓ Solved with 7 moves
Test 8/10: [0, 3, 1, 2]
  ✓ Solved with 6 moves
Test 9/10: [2, 1, 0, 3]
  ✓ Solved with 7 moves
Test 10/10: [2, 3, 1, 0]
  ✓ Solved with 13 moves

Success rate: 100.0%
Passed: 10/10
